<a href="https://colab.research.google.com/github/jonathancagua/NLP/blob/main/EX/Desafio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [20]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [21]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


In [24]:
newsgroups = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

print(f"Número de documentos cargados: {len(newsgroups.data)}")
print("Categorías disponibles:", newsgroups.target_names[:10], "...\n")
print("Ejemplo de documento:\n", newsgroups.data[0][:500])
print("\nCategoría del documento:", newsgroups.target_names[newsgroups.target[0]])

Número de documentos cargados: 11314
Categorías disponibles: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball'] ...

Ejemplo de documento:
 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Categoría del documento: rec.autos


In [25]:
# Instanciamos el vectorizador TF-IDF con eliminación de stopwords
vectorizer = TfidfVectorizer(
    stop_words='english',      # Elimina palabras muy comunes (como "the", "and", etc.)
    max_df=0.95,               # Ignora términos que aparecen en más del 95% de los documentos
    min_df=2,                  # Ignora términos que aparecen en menos de 2 documentos
    max_features=10000,        # Limita el vocabulario a las 10,000 palabras más frecuentes
    ngram_range=(1, 2)         # Incluye unigramas y bigramas
)

# Ajustamos el vectorizador a los datos y transformamos los documentos
X_tfidf = vectorizer.fit_transform(newsgroups.data)

# Mostramos información sobre la matriz TF-IDF
print(f"Dimensiones de la matriz TF-IDF: {X_tfidf.shape}")
print(f"Número de documentos: {X_tfidf.shape[0]}")
print(f"Tamaño del vocabulario: {X_tfidf.shape[1]}")

# Visualizamos una parte del vector del primer documento
print("\nFragmento del vector TF-IDF del primer documento:")
print(X_tfidf[0].toarray()[0][:20])  # Primeras 20 características

Dimensiones de la matriz TF-IDF: (11314, 10000)
Número de documentos: 11314
Tamaño del vocabulario: 10000

Fragmento del vector TF-IDF del primer documento:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [27]:
import random
# Fijamos la semilla para reproducibilidad
random.seed(42)

# Seleccionamos aleatoriamente 5 índices de documentos
selected_indices = random.sample(range(X_tfidf.shape[0]), 5)

# Extraemos los documentos TF-IDF correspondientes
selected_docs = X_tfidf[selected_indices]

# También podrías mostrar las categorías o los textos originales si te interesa
print("📝 Índices seleccionados:", selected_indices)
for i, idx in enumerate(selected_indices):
    category = newsgroups.target_names[newsgroups.target[idx]]
    print(f"\n📄 Documento {i+1} (Índice {idx}, Categoría: {category}):")
    print(newsgroups.data[idx][:300].strip(), "...")

📝 Índices seleccionados: [10476, 1824, 409, 4506, 4012]

📄 Documento 1 (Índice 10476, Categoría: rec.sport.hockey):
This is a general question for US readers:

How extensive is the playoff coverage down there?  In Canada, it is almost
impossible not to watch a series on TV (ie the only two series I have not had
an opportunity to watch this year are Wash-NYI and Chi-Stl, the latter because
I'm in the wrong time zo ...

📄 Documento 2 (Índice 1824, Categoría: comp.sys.mac.hardware):
I think this kind of comparison is pretty useless in general.  The
processor is only good when a good computer is designed around it adn the
computer is used in its designed purpose.  Comparing processor speed is
pretty dumb because all you have to do is just increase the clock speed
to increase ...

📄 Documento 3 (Índice 409, Categoría: comp.graphics):
I can't fiqure this out.  I have properly compiled pov on a unix machine
running SunOS 4.1.3  The problem is that when I run the sample .pov files and
use the 

In [28]:
similarities = cosine_similarity(selected_docs, X_tfidf)

In [29]:
# Calculamos la matriz de similitud de coseno
similarities = cosine_similarity(X_tfidf)

# Iteramos sobre los índices seleccionados
for idx in selected_indices:
    sim = similarities[idx].copy()  # Copiamos para no modificar la matriz original
    sim[idx] = 0  # Evitamos considerar el documento consigo mismo

    # Obtenemos los índices de los 5 documentos más similares
    top_indices = sim.argsort()[-5:][::-1]

    print("\n" + "=" * 100)
    print(f"Documento seleccionado (Índice {idx}) - Categoría: {newsgroups.target_names[newsgroups.target[idx]]}")
    print("-" * 100)
    print(newsgroups.data[idx][:500].strip(), "...\n")

    print("Documentos más similares:\n")
    for i in top_indices:
        categoria = newsgroups.target_names[newsgroups.target[i]]
        similitud = sim[i]
        preview = newsgroups.data[i][:300].replace("\n", " ").strip()

        print(f"Categoría: {categoria} | Similitud: {similitud:.4f}")
        print(f"{preview} ...")
        print("-" * 100)



Documento seleccionado (Índice 10476) - Categoría: rec.sport.hockey
----------------------------------------------------------------------------------------------------
This is a general question for US readers:

How extensive is the playoff coverage down there?  In Canada, it is almost
impossible not to watch a series on TV (ie the only two series I have not had
an opportunity to watch this year are Wash-NYI and Chi-Stl, the latter because
I'm in the wrong time zone!).  We (in Canada) are basically swamped with 
coverage, and I wonder how many series/games are televised nationally or even
locally in the US and how much precedence they take over, say, local new ...

Documentos más similares:

Categoría: rec.sport.hockey | Similitud: 0.2464
I only have one comment on this:  You call this a *classic* playoff year and yet you don't include a Chicago-Detroit series.  C'mon, I'm a Boston fan and I even realize that Chicago-Detroit games are THE most exciting games to watch. ...
-----------

El análisis de similitud de coseno utilizando TF-IDF permitió identificar relaciones temáticas entre documentos, mostrando que aquellos pertenecientes a la misma categoría tienden a compartir mayor similitud, aunque también se detectan coincidencias relevantes entre categorías relacionadas, como en el caso de autos y motocicletas.